# Better Way 46. 내장 알고리즘과 자료구조를 사용하자

* 많은 데이터를 처리하는 파이썬 프로그램을 구현하다 보면 코드의 알고리즘 복잡도 때문에 속도가 떨어지는 현상을 보게 된다..
  * 파이썬이 느리다기 보다는,
  * 최적의 알고리즘과 자료구조를 사용하지 않아서 일어났을 가능성이 크다.

* 파이썬 표준 라이브러리를 사용하자!

### 더블 엔디드 큐
* `collections` 모듈의 `deque` 클래스
  * double-ended queue
  * 큐의 처음과 끝에서 아이템을 삽입하거나 삭제할 때 항상 일정한 시칸이 걸리는 연산을 제공
  * FIFO, First-In-First-Out 큐를 만들 때 이상적

In [1]:
from collections import deque

fifo = deque()
fifo.append(1) # 생산자
x = fifo.popleft() # 소비자

* 내장 타입 `list` 도 큐와 마찬가지로 순서가 있는 아이템 시퀀스를 담는다.
  * 일정한 시간 내에 리스트의 끝에서 아이템을 삽입하거나 삭제할 수 있다.
  * 하지만 리스트의 시작 부분에서 삽입하거나 삭제하는 연산에는 선형적 시간 linear time 이 걸리므로 deque 의 일정한 시간보다 훨씬 느리다.

### 정렬된 딕셔너리
* 표준 딕셔너리는 정렬되어 있지 않다
  * 같은 키와 값을 담은 `dict` 를 순회 해도 다른 순서가 나올 수 있음
  * 이는 딕셔너리의 빠른 해시 테이블을 구현하는 방식이 만들어 낸 뜻밖의 부작용

In [2]:
a = {}
a['foo'] = 1
a['bar'] = 2

In [3]:
from random import randint

# ? while True 로 했더니 무한루프에 빠짐.
for _ in range(100000):
    z = randint(99, 1013)
    b = {}
    
    b['foo'] = 1
    
    for i in range(z):
        b[i] = i
    
    b['bar'] = 2
    
    for i in range(z):
        del b[i]
        
    if str(b) != str(a):
        break
        
print(a)
print(b)
print('Equal?', a==b)

{'foo': 1, 'bar': 2}
{'foo': 1, 'bar': 2}
Equal? True


* `collections` 모듈의 `OrderedDict` 클래스는 키가 삽입된 순서를 유지하는 특별한 딕셔너리

In [4]:
from collections import OrderedDict

a = OrderedDict()
a['foo'] = 1
a['bar'] = 2

b = OrderedDict()
b['foo'] = 'red'
b['bar'] = 'blue'

for value1, value2 in zip(a.values(), b.values()):
    print(value1, value2)

1 red
2 blue


### 기본 딕셔너리 (defaultdict)

* 딕셔너리는 통계를 관리하고 추적하는 작업에 유용
  * 하지만, 어떤 키가 이미 존재한다고 가정할 수 없음
  * 간단한 작업도 까다로워지는 이유

In [5]:
stats = {}
key = 'my_counter'

if key not in stats:
    stats[key] = 0

stats[key] += 1

* `collections` 모듈의 `defaultdict` 클래스
  * 키가 존재하지 않으면 자동으로 기본값을 저장도록 하여 작업을 간소화시킨다
  * 키가 없을 때마다 기본값을 반환할 함수를 제공하기만 하면 됨

In [6]:
from collections import defaultdict

stats = defaultdict(int) # int = 0을 반환하는 내장함수 - BetterWay 23 참고
stats['my_counter'] += 1

### 힙큐
* 힙 heap 은 우선순위 큐 priority queue 를 유지하는 유용한 자료구조
  * `heapq` 모듈은 표준 `list` 타입으로 힙을 생성하는 `heappush`, `heappop`, `nsmallest` 같은 함수를 제공한다.
  * 임의의 우선순위를 가지는 아이템은 어떤 순서로도 힙에 삽입할 수 있다

In [7]:
from heapq import heappush, heappop, nsmallest

a = []

heappush(a, 5)
heappush(a, 3)
heappush(a, 7)
heappush(a, 4)

print(a)

[3, 4, 7, 5]


* 아이템은 가장 우선순위가 높은 것 (가장 낮은 수) 부터 제거된다

In [8]:
print(heappop(a), heappop(a), heappop(a), heappop(a))

3 4 5 7


* 결과로 만들어지는 `list` (코드에서는 `a`) 를 `heapq` 외부에서도 쉽게 사용할 수 있다.
  * 힙의 0 인덱스에 접근하면 항상 가장 작은 아이템이 반환된다.

In [9]:
a = []

heappush(a, 5)
heappush(a, 3)
heappush(a, 7)
heappush(a, 4)

assert a[0] == nsmallest(1, a)[0] == 3

* `list` 의 `sort` 메서드를 호출하더라도 힙의 불변성이 유지된다.

In [10]:
print('Before: ', a)
a.sort()
print('After:  ', a)

Before:  [3, 4, 7, 5]
After:   [3, 4, 5, 7]


* 이러한 각 `heapq` 연산에 걸리는 시간은 리스트의 길이에 비례하여 로그 형태로 증가한다.
  * `list` 로 같은 동작을 수행하면 선형적으로 증가한다.

### 바이섹션
* `list` 에서 아이템을 검색하는 작업은 `index` 메서드를 호출할 때 리스트의 길이에 비례한 선형적 시간이 걸린다.

In [11]:
x = list(range(10**6))
i = x.index(991234)
print(i)

991234


In [12]:
# test 1
x = list(range(100000))

%timeit i = x.index(91234)

1.48 ms ± 76.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [13]:
# test 2
x = list(range(200000))

%timeit i = x.index(191234)

3.19 ms ± 49.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [14]:
# test 3
x = list(range(300000))

%timeit i = x.index(291234)

4.89 ms ± 57.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


* `bisect_left` 같은 `bisect` 모듈의 함수는 **정렬된 아이템 시퀀스**를 대상으로 한 효율적인 바이너리 검색을 제공한다.
  * `bisect_left` 가 반환하는 인덱스는 시퀀스에 들어간 값의 삽입 지점이다

In [15]:
from bisect import bisect_left

i = bisect_left(x, 991234)
print(i)

300000


In [16]:
# test 1
x = list(range(100000))

%timeit i = bisect_left(x, 91234)

579 ns ± 9.05 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [17]:
# test 2
x = list(range(200000))

%timeit i = bisect_left(x, 191234)

662 ns ± 37.9 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [18]:
# test 3
x = list(range(300000))

%timeit i = bisect_left(x, 291234)

621 ns ± 2.32 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


* 바이너리 검색의 복잡도는 로그 형태로 증가
  * 아이템 백만개를 담은 리스트를 `bisect` 로 검색할 때 걸리는 시간과 아이템 14개를 담은 리스트를 `index` 로 순차 검색할 때 걸리는 시간이 거의 같음

### 이터레이터 도구
* 내장 모듈 `itertools` 는 이터레이터를 구성하거나 이터레이터와 상호작용 하는데 유용한 함수를 다수 포함한다
  * 이터레이터는 BetterWay 16, 17 참조
* 파이썬 2에서는 이런 기능을 모두 이용하진 못하지만, 모듈 문서에 있는 예제를 참고하면 간단하게 만들 수 있다

In [19]:
import itertools

help(itertools)

Help on built-in module itertools:

NAME
    itertools - Functional tools for creating and using iterators.

DESCRIPTION
    Infinite iterators:
    count(start=0, step=1) --> start, start+step, start+2*step, ...
    cycle(p) --> p0, p1, ... plast, p0, p1, ...
    repeat(elem [,n]) --> elem, elem, elem, ... endlessly or up to n times
    
    Iterators terminating on the shortest input sequence:
    accumulate(p[, func]) --> p0, p0+p1, p0+p1+p2
    chain(p, q, ...) --> p0, p1, ... plast, q0, q1, ... 
    chain.from_iterable([p, q, ...]) --> p0, p1, ... plast, q0, q1, ... 
    compress(data, selectors) --> (d[0] if s[0]), (d[1] if s[1]), ...
    dropwhile(pred, seq) --> seq[n], seq[n+1], starting when pred fails
    groupby(iterable[, keyfunc]) --> sub-iterators grouped by value of keyfunc(v)
    filterfalse(pred, seq) --> elements of seq where pred(elem) is False
    islice(seq, [start,] stop [, step]) --> elements from
           seq[start:stop:step]
    starmap(fun, seq) --> fun(*seq

* `itertools` 에 있는 함수는 크게 세 가지 카테고리
  * 이터레이터 연결
    * `chain`: 여러 이터레이터를 순차적인 이터레이터 하나로 결합한다.
    * `cycle`: 이터레이터의 아이템을 영원히 반복한다.
    * `tee`: 이터레이터 하나를 병렬 이터레이터 여러 개로 나눈다.
    * `zip_longest`: 길이가 서로 다른 이터레이터들에도 잘 동작하는 내장 함수 `zip` 의 변형
  * 이터레이터에서 아이템 필터링
    * `isslice`: 복사 없이 이터레이터를 숫자로 된 인덱스로 슬라이스한다.
    * `takewhile`: predicate function 이 `True`를 반환하는 동안 이터레이터의 아이템을 반환한다.
    * `dropwhile`: predicate function 이 처음으로 `False` 를 반환하고 나면 이터레이터의 아이템을 반환한다.
    * `filterfalse`: predicate function 이 `False` 를 반환하는 이터레이터의 모든 아이템을 반환한다. 내장함수 `filter` 의 반대 기능을 한다.
  * 이터레이터에 있는 아이템들의 조합
    * `product`: 이터레이터에 있는 아이템들의 카테시안 곱을 반환한다. 깊게 중첩된 list comprehension 에 대한 훌륭한 대안.
    * `permutations`:  이터레이터에 있는 아이템을 담은 길이 N 의 순서 있는 순열을 반환한다.
    * `combinations`: 이터레이터에 있는 아이템을 중복되지 않게 담은 길이 N 의 순서 없는 조합을 반환한다.

* 설명하지 않은 `itertools` 모듈의 기능과 사용방법도 많음
  * 까다로운 이터레이션 코드를 다루는 상황이라면 `itertools` 문서를 다시 살펴보고 쓸만한 기능이 있는지 찾아볼 가치가 있다.

### 핵심 정리
* 알고리즘과 자료 구조를 표현하는 파이썬의 내장 모듈을 사용하자
* 이 기능들을 직접 재구현하지는 말자. 올바르게 만들기가 어렵기 때문이다

# Better Way 47. 정밀도가 중요할 때는 decimal 을 사용하자

* 파이썬은 숫자 데이터를 다루는 코드를 작성하기에 아주 뛰어난 ^^ 언어
  * 정수 타입: 현실적인 크기의 값을 모두 표현할 수 있다
  * 배정밀도 부동 소수점: IEEE 754 표준을 따른다
  * 허수 값을 표현하는 표준 복소수 타입도 제공한다
* 그러나 이것만으로는 모든 상황을 충족하지는 못한다!

* 예시) 고객에게 부과할 국제 전화 요금을 계산
  * 고객이 몇 분간, 몇 초간 통화했는지 알고 있음 (3분 42초)
  * 미국에서 남극 대륙을 건너 통화했을 때의 요금도 정해져있음 (분당 1.45 달러)

In [20]:
rate = 1.45
seconds = 3 * 60 + 42
cost = rate * seconds / 60
print(cost)

5.364999999999999


* 센트 단위에 맞춰 반올림한다고 하자.
  * 근데 이 경우, 값을 버림!

In [21]:
print(round(cost, 2))

5.36


* 연결 비용이 훨씬 저렴한 곳에서 일어나는 아주 짧은 통화도 지원해야 한다고 해보자.
  * 분당 0.05달러, 5초 동안 일어난 통화

In [22]:
rate = 0.05
seconds = 5
cost = rate * seconds / 60
print(cost)

0.004166666666666667


* 결과가 너무 작아서 반올림하면 0이 된다.

In [23]:
print(round(cost, 2))

0.0


* 내장 모듈 `decimal` 의 `Decimal` 클래스를 사용하자
  * 기본적으로 소수점이 28자리인 고정 소수점 연산을 제공한다
  * 필요하면 자릿수를 더 늘릴 수 있다
  * IEEE 754 부동 소수점 수의 정확도 문제도 피해갈 수 있다
  * 반올림 연산을 더 세밀하게 제어할 수 있다

In [24]:
from decimal import Decimal, ROUND_UP

rate = Decimal('1.45')
seconds = Decimal('222') # 3 * 60 + 42
cost = rate * seconds / Decimal('60')
print(cost)

5.365


* `Decimal` 클래스에는 원하는 반올림 동작에 따라 필요한 소수점 위치로 정확하게 반올림하는 내장 함수가 있다
  * 참고: 코드에서는 올림하고 있음. 반올림을 사용하려면 `ROUND_HALF_UP` 사용.

In [25]:
rounded = cost.quantize(Decimal('0.01'), rounding=ROUND_UP)
print(rounded)

5.37


* 이 방식으로 `quantize` 메서드를 사용하면 짧고 저렴한 통화에 해당하는 적은 통화료도 적절하게 처리할 수 있다

In [26]:
rate = Decimal('0.05')
seconds = Decimal('5')
cost = rate * seconds / Decimal('60')
print(cost)

0.004166666666666666666666666667


In [27]:
rounded = cost.quantize(Decimal('0.01'), rounding=ROUND_UP)
print(rounded)

0.01


* `Decimal` 이 고정 소수점 수에도 잘 동작하지만 아직도 정확도 면에서는 제약이 있다.
  * 1 / 3 은 근삿값으로 계산됨 = `Decimal('0.3333333333333333333333333333')`
  * 정확도에 제한이 없는 유리수를 표현하려면 `fractions` 의 `Fraction` 클래스를 사용해야 한다.

### 핵심 정리
* 파이썬은 현실적으로 모든 유형의 숫자 값을 표현할 수 있는 내장 타입과 클래스를 모듈로 제공한다.
* `Decimal` 클래스는 화폐 연산처럼 정밀도가 높고 정확한 반올림이 필요한 상황에 안성 맞춤이다.

# Better Way 48. 커뮤니티에서 만든 모듈을 어디서 찾아야 하는지 알아두자

* 파이썬 모듈들의 중앙 저장소 https://pypi.python.org
  * 여러 파이썬 커뮤니티 (혹은 개인?) 에서 모듈을 만들고 관리한다
  * 이러한 커뮤니티에 익숙하지 않다면 파이썬 패키지 인덱스 PyPI 가 원하는 목적에 맞는 코드를 찾을 수 있는 좋은 방법

* 패키지 인덱스를 사용하려면 `pip` 을 사용
  * 파이썬 3.4 와 이후 버전에는 기본으로 설치되어 있음
  * 이전 버전은 https://packaging.python.org 에서 `pip` 설치 명령서를 찾아보면 된다
* 예시) `pip` 로 새 모듈 설치하기

In [28]:
! pip3 install pytz

  Using cached https://files.pythonhosted.org/packages/e7/f9/f0b53f88060247251bf481fa6ea62cd0d25bf1b11a87888e53ce5b7c8ad2/pytz-2019.3-py2.py3-none-any.whl


* 파이썬 3 패키지 설치할 땐 `pip3`, 파이썬 2 패키지 설치할 땐 `pip`
* `pip` 를 `pyenv` 와 함께 사용하면 프로젝트에 설치할  패키지 집합을 추적할 수 있다
  * Better Way 53. 의존성을 분리하고 재현하려면 가상환경을 사용하자

* PyPI 의 각 모듈에는 자체의 소프트웨어 라이선스가 있다.
  * 대부분의 패키지는 무료 또는 오픈소스
  * 대부분의 경우 이런 라이선스는 프로그램에 모듈의 복사본을 포함하는 것을 허용한다
    * 의심이 들면 변호사와 상의하자

### 핵심 정리
* 파이썬 패키지 인덱스 PyPI 는 파이썬 커뮤니티에서 만들고 유지하는 풍부한 공통 패키지를 포함하고 있다
* `pip` 는 PyPI 에서 패키지를 설치하는 데 사용하는 명령줄 도구이다
* `pip` 는 파이썬 3.4 와 그 이후 버전에는 기본으로 설치되어 있다. 이전 버전에서는 직접 설치해야 한다.
* PyPI 모듈의 대부분은 무료이자 오픈 소스 소프트웨어이다.